In [2]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
# @assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
pkgs = [
    "Revise",
    # "FASTX",
    # "StatsBase",
    # "Distributions",
    # "StatsPlots",
    # "Random",
    # "Dates",
    "DataFrames",
    # "BioSequences",
    # "Conda",
    # "Downloads"
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
# Pkg.develop(path="/global/cfs/projectdirs/m4269/cjprybol/Mycelia")
# Pkg.develop(path="../../..")
import Mycelia

[ Info: Precompiling Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd]
[ Info: Skipping precompilation since __precompile__(false). Importing Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd].


In [3]:
PROJECT_BASEDIR = dirname(pwd())
data_dir = joinpath(PROJECT_BASEDIR, "data")
genome_dir = mkpath(joinpath(data_dir, "genomes"))

"/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes"

In [15]:
reference_fastas

6-element Vector{String}:
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/fF3EHHs.fna"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/3NEzu1DmBy.fna"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/GCF_001610725.1.fna"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/19tEIXfKKdzr.fna"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/GCF_024820135.1.fna"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/GCF_023169545.1.fna"

In [18]:
reference_fastas = sort(filter(x -> occursin(r"\.fna$", x) && !occursin("normalized", x), readdir(genome_dir, join=true)), by=x->filesize(x))
for reference_fasta in reference_fastas
    if !isdir(reference_fasta * "_RTG")
        run(`$(Mycelia.MAMBA) run --live-stream -n rtg-tools rtg format -o $(reference_fasta)_RTG $(reference_fasta)`)
    end
end
identifier_to_reference_fasta = Dict(first(split(basename(x), '.')) => x for x in reference_fastas)
reference_variants = sort(filter(x -> occursin(r"fna\.normalized\.vcf\.gz$", x), readdir(genome_dir, join=true)), by=x->filesize(x))

6-element Vector{String}:
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/fF3EHHs.fna.normalized.vcf.gz"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/3NEzu1DmBy.fna.normalized.vcf.gz"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/GCF_001610725.1.fna.normalized.vcf.gz"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/19tEIXfKKdzr.fna.normalized.vcf.gz"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/GCF_024820135.1.fna.normalized.vcf.gz"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/GCF_023169545.1.fna.normalized.vcf.gz"

In [19]:
identifiers = [first(split(basename(x), '.')) for x in reference_variants]
identifier_to_reference_variants = Dict(first(split(basename(x), '.')) => x for x in reference_variants)

Dict{SubString{String}, String} with 6 entries:
  "GCF_024820135" => "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant…
  "GCF_023169545" => "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant…
  "3NEzu1DmBy"    => "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant…
  "GCF_001610725" => "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant…
  "19tEIXfKKdzr"  => "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant…
  "fF3EHHs"       => "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant…

In [20]:


cactus_dirs = filter(x -> occursin(r"cactus"i, x) && isdir(x), readdir(genome_dir, join=true))
pggb_dirs = filter(x -> occursin(r"pggb"i, x), readdir(genome_dir, join=true))
clair3_dirs = filter(x -> occursin(r"clair3$"i, x) && isdir(x), readdir(genome_dir, join=true))
other_vcfs = filter(x -> occursin(r"vcf$", x) && isfile(x) && (occursin(r"bcftools", x) || occursin(r"freebayes", x)) && !occursin(r"\.sorted\.vcf$", x), readdir(genome_dir, join=true))
clair3_vcfs = joinpath.(clair3_dirs, "merge_output.vcf.gz")
pggb_vcfs = filter(x -> occursin(r"\.vcf$", x), reduce(vcat, readdir.(pggb_dirs, join=true)))
cactus_vcfs = filter(x -> occursin(r"cactus\.vcf\.gz$", x), reduce(vcat, readdir.(cactus_dirs, join=true)))
vcfs = vcat(cactus_vcfs, pggb_vcfs, clair3_vcfs, other_vcfs)

127-element Vector{String}:
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/19tEIXfKKdzr-cactus/19tEIXfKKdzr-cactus.vcf.gz"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/3NEzu1DmBy-cactus/3NEzu1DmBy-cactus.vcf.gz"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/GCF_001610725-cactus/GCF_001610725-cactus.vcf.gz"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/GCF_023169545-cactus/GCF_023169545-cactus.vcf.gz"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/GCF_024820135-cactus/GCF_024820135-cactus.vcf.gz"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/fF3EHHs-cactus/fF3EHHs-cactus.vcf.gz"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/19tEIXfKKdzr.fna__19tEIXfKKdzr.fna

In [21]:
vcfs = filter(x -> !occursin("fixed", x), vcfs)

120-element Vector{String}:
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/19tEIXfKKdzr-cactus/19tEIXfKKdzr-cactus.vcf.gz"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/3NEzu1DmBy-cactus/3NEzu1DmBy-cactus.vcf.gz"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/GCF_001610725-cactus/GCF_001610725-cactus.vcf.gz"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/GCF_023169545-cactus/GCF_023169545-cactus.vcf.gz"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/GCF_024820135-cactus/GCF_024820135-cactus.vcf.gz"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/fF3EHHs-cactus/fF3EHHs-cactus.vcf.gz"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/19tEIXfKKdzr.fna__19tEIXfKKdzr.fna

In [22]:
# make a table that lists calling method, identifier, file path

In [23]:
vcf_table = DataFrames.DataFrame(
    identifier = String[],
    variant_caller = String[],
    coverage = String[],
    vcf = String[]
)
for vcf in vcfs
    identifier = first(filter(identifier -> occursin(identifier, vcf), identifiers))
    variant_callers = [
        "cactus",
        "PGGB",
        "bcftools",
        "freebayes",
        "clair3"
    ]
    variant_caller = first(filter(variant_caller -> occursin(variant_caller, vcf), variant_callers))
    if variant_caller in ["cactus", "PGGB"]
        coverage = "N/A"
    else
        if occursin("10x", vcf)
            coverage = "10x"
        elseif occursin("100x", vcf)
            coverage = "100x"
        elseif occursin("1000x", vcf)
            coverage = "1000x"
        end
    end
    push!(vcf_table, (;identifier, variant_caller, coverage, vcf))
end
vcf_table

Row,identifier,variant_caller,coverage,vcf
,String,String,String,String
1,19tEIXfKKdzr,cactus,N/A,/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/19tEIXfKKdzr-cactus/19tEIXfKKdzr-cactus.vcf.gz
2,3NEzu1DmBy,cactus,N/A,/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/3NEzu1DmBy-cactus/3NEzu1DmBy-cactus.vcf.gz
3,GCF_001610725,cactus,N/A,/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/GCF_001610725-cactus/GCF_001610725-cactus.vcf.gz
4,GCF_023169545,cactus,N/A,/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/GCF_023169545-cactus/GCF_023169545-cactus.vcf.gz
5,GCF_024820135,cactus,N/A,/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/GCF_024820135-cactus/GCF_024820135-cactus.vcf.gz
6,fF3EHHs,cactus,N/A,/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/fF3EHHs-cactus/fF3EHHs-cactus.vcf.gz
7,19tEIXfKKdzr,PGGB,N/A,/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/19tEIXfKKdzr.fna__19tEIXfKKdzr.fna.normalized.vcf.fna.joint.fna__PGGB/19tEIXfKKdzr.fna__19tEIXfKKdzr.fna.normalized.vcf.fna.joint.fna.e316b50.417fcdf.db7e83b.smooth.final.gfa.vcf
8,3NEzu1DmBy,PGGB,N/A,/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/3NEzu1DmBy.fna__3NEzu1DmBy.fna.normalized.vcf.fna.joint.fna__PGGB/3NEzu1DmBy.fna__3NEzu1DmBy.fna.normalized.vcf.fna.joint.fna.e316b50.417fcdf.db7e83b.smooth.final.gfa.vcf
9,GCF_001610725,PGGB,N/A,/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/GCF_001610725.1.fna__GCF_001610725.1.fna.normalized.vcf.fna.joint.fna__PGGB/GCF_001610725.1.fna__GCF_001610725.1.fna.normalized.vcf.fna.joint.fna.e316b50.417fcdf.db7e83b.smooth.final.gfa.vcf


In [24]:
function fix_vcf(;vcf, outvcf = replace(vcf, r"\.vcf$" => ".fixed.vcf"))
    open(outvcf, "w") do io
        for line in eachline(vcf)
            if occursin(r"^##", line)
                println(io, line)
            elseif occursin(r"^#", line)
                println(io, line * "\tsample")
            else
                # println(io, replace(line, r"\s+" => "\t"))
                println(io, line * "\t1")
            end
        end
    end
    return outvcf
end

fix_vcf (generic function with 1 method)

In [9]:
# # skipped PGGB - need to rerun



# indices = findall(vcf_table[!, "variant_caller"] .== "PGGB")
# # 3, 4, 5
# for row in DataFrames.eachrow(vcf_table[indices[7:end], :])
#     vcf = row["vcf"]
#     reference_fasta = identifier_to_reference_fasta[row["identifier"]]
#     baseline_variants = identifier_to_reference_variants[row["identifier"]]
#     fixed_vcf = fix_vcf(vcf=vcf)
#     processed_vcf = Mycelia.preprocess_vcf(vcf=fixed_vcf, fasta=reference_fasta)
#     outdir = joinpath(baseline_variants * "_RTG", basename(processed_vcf))
#     if !isfile(processed_vcf * ".tbi")
#         @show filesize(processed_vcf)
#         run(`$(Mycelia.MAMBA) run --live-stream -n rtg-tools rtg index --format vcf $(processed_vcf)`)
#     end
#     if !isdir(outdir)
#         run(`$(Mycelia.MAMBA) run --live-stream -n rtg-tools rtg RTG_MEM=8G vcfeval --all-records --vcf-score-field QUAL --threads=1 --template $(reference_fasta)_RTG --baseline $(baseline_variants) --calls $(processed_vcf) --squash-ploidy --output $(outdir)`)
#         non_snp_roc = joinpath(outdir, "non_snp_roc.tsv.gz")
#         snp_roc = joinpath(outdir, "snp_roc.tsv.gz")
#         weighted_roc = joinpath(outdir, "weighted_roc.tsv.gz")
#         roc_png = joinpath(outdir, "roc.png")
#         roc_svg = joinpath(outdir, "roc.svg")
#         run(`$(Mycelia.MAMBA) run --live-stream -n rtg-tools rtg RTG_MEM=8G rocplot --png $(roc_png) --svg $(roc_svg) --curve $(non_snp_roc)=non_snp_roc --curve $(snp_roc)=snp_roc --curve $(weighted_roc)=weighted_roc `)
#     else
#         @info "$(outdir) already exists"
#     end
# end

In [28]:
outdirs = String[]
for row in DataFrames.eachrow(vcf_table)
    vcf = row["vcf"]
    reference_fasta = identifier_to_reference_fasta[row["identifier"]]
    baseline_variants = identifier_to_reference_variants[row["identifier"]]
    if row["variant_caller"] == "PGGB"
        vcf = fix_vcf(vcf=vcf)
        processed_vcf = replace(vcf, r"\.vcf(?:\.gz)?$" => ".sorted.normalized.vcf.gz")
        outdir = joinpath(baseline_variants * "_RTG", basename(processed_vcf))
    elseif row["variant_caller"] == "clair3"
        processed_vcf = replace(vcf, r"\.vcf(?:\.gz)?$" => ".sorted.normalized.vcf.gz")
        outdir = joinpath(baseline_variants * "_RTG", basename(dirname(processed_vcf)))
    else
        processed_vcf = replace(vcf, r"\.vcf(?:\.gz)?$" => ".sorted.normalized.vcf.gz")
        outdir = joinpath(baseline_variants * "_RTG", basename(processed_vcf))
    end
    push!(outdirs, outdir)
    if isdir(outdir)
        rm(outdir, recursive=true)
    end
    # if !isdir(outdir)
    try
        processed_vcf = Mycelia.preprocess_vcf(vcf=vcf, fasta=reference_fasta)
        if !isfile(processed_vcf * ".tbi")
            @show filesize(processed_vcf)
            run(`$(Mycelia.MAMBA) run --live-stream -n rtg-tools rtg index --format vcf $(processed_vcf)`)
        end
        run(`$(Mycelia.MAMBA) run --live-stream -n rtg-tools rtg RTG_MEM=8G vcfeval --all-records --vcf-score-field QUAL --threads=1 --template $(reference_fasta)_RTG --baseline $(baseline_variants) --calls $(processed_vcf) --squash-ploidy --output $(outdir)`)
        non_snp_roc = joinpath(outdir, "non_snp_roc.tsv.gz")
        snp_roc = joinpath(outdir, "snp_roc.tsv.gz")
        weighted_roc = joinpath(outdir, "weighted_roc.tsv.gz")
        roc_png = joinpath(outdir, "roc.png")
        roc_svg = joinpath(outdir, "roc.svg")
        run(`$(Mycelia.MAMBA) run --live-stream -n rtg-tools rtg RTG_MEM=8G rocplot --png $(roc_png) --svg $(roc_svg) --curve $(non_snp_roc)=non_snp_roc --curve $(snp_roc)=snp_roc --curve $(weighted_roc)=weighted_roc `)
    catch e
        println(e)
    end
        # else
    #     @info "$(outdir) already exists"
    # end
end
outdirs

Writing to /tmp/bcftools.bncOI0
Merging 1 temporary files
Cleaning
Done
Lines   total/split/joined/realigned/skipped:	1257/0/0/105/0


Selected score threshold using: maximized F-measure
Threshold  True-pos-baseline  True-pos-call  False-pos  False-neg  Precision  Sensitivity  F-measure
----------------------------------------------------------------------------------------------------
   60.000                315           1257          0          0     1.0000       1.0000     1.0000
     None                315           1257          0          0     1.0000       1.0000     1.0000



Writing to /tmp/bcftools.JEbHwn
Merging 1 temporary files
Cleaning
Done
Lines   total/split/joined/realigned/skipped:	266/0/0/19/0


Selected score threshold using: maximized F-measure
Threshold  True-pos-baseline  True-pos-call  False-pos  False-neg  Precision  Sensitivity  F-measure
----------------------------------------------------------------------------------------------------
   60.000                100            266          0          0     1.0000       1.0000     1.0000
     None                100            266          0          0     1.0000       1.0000     1.0000



Writing to /tmp/bcftools.NK8d0Y
Merging 1 temporary files
Cleaning
Done
Lines   total/split/joined/realigned/skipped:	299/0/0/24/0


...

Writing to /tmp/bcftools.J96Ubb
Merging 1 temporary files
Cleaning
Done
Lines   total/split/joined/realigned/skipped:	67/0/0/0/0


Selected score threshold using: maximized F-measure
Threshold  True-pos-baseline  True-pos-call  False-pos  False-neg  Precision  Sensitivity  F-measure
----------------------------------------------------------------------------------------------------
   41.586                 18             40         22         12     0.6452       0.6000     0.6218
     None                 18             40         27         12     0.5970       0.6000     0.5985



Writing to /tmp/bcftools.Jylfvm
Merging 1 temporary files
Cleaning
Done
Lines   total/split/joined/realigned/skipped:	31/0/0/14/0


Selected score threshold using: maximized F-measure
Threshold  True-pos-baseline  True-pos-call  False-pos  False-neg  Precision  Sensitivity  F-measure
----------------------------------------------------------------------------------------------------
    1.593                 30             31          0          0     1.0000       1.0000     1.0000
     None                 30             31          0          0     1.0000       1.0000     1.0000



120-element Vector{String}:
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/19tEIXfKKdzr.fna.normalized.vcf.gz_RTG/19tEIXfKKdzr-cactus.sorted.normalized.vcf.gz"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/3NEzu1DmBy.fna.normalized.vcf.gz_RTG/3NEzu1DmBy-cactus.sorted.normalized.vcf.gz"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/GCF_001610725.1.fna.normalized.vcf.gz_RTG/GCF_001610725-cactus.sorted.normalized.vcf.gz"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/GCF_023169545.1.fna.normalized.vcf.gz_RTG/GCF_023169545-cactus.sorted.normalized.vcf.gz"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/GCF_024820135.1.fna.normalized.vcf.gz_RTG/GCF_024820135-cactus.sorted.normalized.vcf.gz"
 "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-b

In [30]:
count([isdir(dir) for dir in outdirs])

117

Performance Metrics: Start by calculating key metrics for each variant calling method, including precision (positive predictive value), recall (sensitivity), accuracy, and F1 score. These metrics will provide a comprehensive view of each method's performance.

Confusion Matrix: For each method, create a confusion matrix that shows the number of true positives, false positives, true negatives, and false negatives. This will give you a clear picture of the type of errors made by each method.

Receiver Operating Characteristic (ROC) Curve: Plot ROC curves for each method. This plot shows the trade-off between true positive rate and false positive rate and is useful for comparing the performance of different methods.

Precision-Recall Curve: Especially when dealing with imbalanced datasets (where variants are rare), precision-recall curves can be more informative than ROC curves.

Heatmaps: Use heatmaps to visualize the performance metrics across different genomes and depths of coverage. This can provide an intuitive understanding of how performance varies with these factors.

jaccard similarity?
AUC